In [2]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

# Lecture 10

In this lecture, we will:
1. Introduce python Lists
2. Show how to build tables from rows instead of columns
3. finish our discussion on grouping
4. Introduce pivots -- a special kind of grouping
5. Introduce joins

---

## Lists

Lists in python are like arrays but they can contain things of different types and you make them by using square brackets:

In [3]:
make_array(1,5,'hello', 5.0)

array(['1', '5', 'hello', '5.0'],
      dtype='<U32')

In [4]:
[1, 5, 'hello', 5.0]

[1, 5, 'hello', 5.0]

In [5]:
another_list = [1, 5, 'hello', 5.0, make_array(1,2,3)]
another_list

[1, 5, 'hello', 5.0, array([1, 2, 3])]

In [6]:
another_list = [another_list, 1]
len(another_list)
another_list

[[1, 5, 'hello', 5.0, array([1, 2, 3])], 1]

You have to use square brakets when accessing elements from a list (you can't use item).

In [7]:
make_array(1,5,'hello', 5.0).item(0) # .item() is exclusive to arrays

'1'

In [8]:
another_list[2] # when we want to access an element in a list, we use []

IndexError: list index out of range

In [ ]:
another_list[1]

Recall that for arrays you could do either. (In the real world, most people would probably use square brackets).

In [ ]:
my_array = make_array("Apple", "Banana", "Corn")
my_array[1]

Same thing as:

In [ ]:
my_array.item(1)

### Rows from lists

We need lists to be able to add a row to our table.  Why?

Here we make a table with a column (we have been doing this all semester).

In [ ]:
Table().with_columns('Numbers', [1, 2, 3])
# Table() initializes a table
# .with_columns # adds new columns 

Here we make an empty table with just column labels. (This is new!)

In [ ]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

Here we add rows to our table.  Each row is a `List` of elements -- one for each column.

In [ ]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

---

<center> Return to Slides </center>

---

## Grouping

For this part of the notebook we will use the following toy data:

In [ ]:
cones = Table.read_table('data/cones.csv')
cones

**Exercise:** Use the `group` function to determine the number of cones with each flavor.

In [ ]:
cones.group('Flavor')

<details> <summary>Click for Solution</summary>

```python
cones.group('Flavor')
```
</details>

**Exercise:** Use the `group` function to compute the average price of cones for each flavor.

In [ ]:
cones.group('Flavor',np.average)

<details> <summary>Click for Solution</summary> 

```python
cones.group('Flavor', np.average)
 ```
</details>

**Exercise:** Use the `group` function to compute min price of cones for each flavor.

In [ ]:
cones.group('Flavor',min)

In [ ]:
cones

In [ ]:
# categorize by flavor AND color
cones.group(['Flavor', 'Color'],max)

**Question**:
Why does color have a min?

<details> <summary>Click for Solution</summary>

```python
cones.group('Flavor', np.min)
```

---

<center> Return to Slides </center>

---

## Review Grouping Using the Welcome Survey

Now let's use the group function to study you:

In [ ]:
survey = Table.read_table("data/classdatasurvey_w24.csv") # changed file path name to our current quarter's survey
survey.show(3)

**Exercise:** What happens if we group by `"Year"` with no additional arguments?

In [ ]:
survey.group("What year are you in?")

What is the relationship between year and the average amount of sleep?

**Exercise:** Computer the average `"Hours of sleep"` for each `"Year"` and display the result as bar chart with the bars in a meaningful order.

In [ ]:
survey_cleaned = survey.where("What year are you in?", are.not_equal_to('nan'))
survey_cleaned.group('What year are you in?')
survey_cleaned.group('How many hours of sleep do you get?').show()


**Excercise**: Use the function below to convert the values in the 'year' column to strings that sort meaninfully. (use apply)

In [ ]:
def convert_year_string(y):
    if y == 'First Year':
        return 'Year 1'
    elif y == 'Second Year':
        return 'Year 2'
    elif y == 'Third Year':
        return 'Year 3'
    elif y == 'Fourth Year':
        return 'Year 4'
    elif y == 'Firth Year or above':
        return 'Year 5+'
    else:
        return 'graduate'

<details><summary>Click for Solution</summary>
    
```python
survey['year_'] = survey.apply(convert_year_string, 'Year')
survey.select('year_', 'Hours of sleep').group('year_', np.average).barh('year_')
plots.xlim([6,7.5])
```
    
</details>

### Understanding Extraversion

How extraverted is the class?

In [ ]:
survey.hist('Extraversion')

**Exercise:** Compute the *average* `"Number of textees"`, `"Hours of sleep"`, `"Piercings"` for each value of `"Extraversion"`.  Save the resulting table as `by_extra`.

In [ ]:
by_extra = 

<details><summary>Click for Solution</summary>
    
```python
by_extra = (
    survey
    .select("Extraversion", "Number of textees", "Hours of sleep", "Piercings")
    .group('Extraversion', np.mean)
)
by_extra
```
    
</details>

Do you see any trends?  Let's plot the data to make it clearer.

In [ ]:
by_extra.plot("Extraversion", "Number of textees mean", marker="*")

In [ ]:
by_extra.plot("Extraversion", "Hours of sleep mean", marker="*")

---

<center> Return to Slides </center>

---

## Grouping by Two Columns 

Do right-handed people tend to sleep on their left side and left-handed people sleep on their right side? 

**Exercise:** Use the group function to count how many people are in each combination of `"Handedness"` and `"Sleep position"`.

<details><summary>Click for Solution</summary> 
    
```python
(
    survey
    .group(column_or_label=["Handedness", "Sleep position"])
    .show()
)
```
</details>

What combination of `"Handedness"` and `"Sleep position"` results in the most sleep on average?

**Exercise:** Use group to answer the above question.

<details><summary>Click for Solution</summary>
    
```python
(
    survey
    .select("Handedness", "Sleep position", "Hours of sleep")
    .group(['Handedness', 'Sleep position'], np.average)
    .show()
)
```
    
</details>

## Pivot Tables

When grouping by two columns it will sometimes be easier to re-arrange the table so the columns correspond to the *values* of one of the original grouped columns. 

Here is an example where I construct a pivot table that is equivalent to the above two column groupby operation:

```python
(
    survey
    .select("Handedness", "Sleep position", "Hours of sleep")
    .group(["Handedness", "Sleep position"], np.average)
    .show()
)
```

Here I am rewriting the example by explicity naming the function arguments.  You wouldn't do this in practice but it helps to highlight the meaning of the four things we passed to the function above.

In [ ]:
(
    survey
    .pivot(
        columns = "Handedness", # The column whose unique values become columns
        rows    = "Sleep position", # The column whose unique values become rows
        values  = "Hours of sleep", # The column that will be used to compute the cells
        collect = np.average # How to aggregate the values 
    )
)

Just as with the regular `group` function you will often want counts of combinations. In this case, you only need to specify the column to turn into columns and the column to turn into rows.

In [ ]:
survey.pivot("Sleep position", "Handedness")

Pivots can be useful for visualizations:

In [ ]:
survey.pivot("Sleep position", "Handedness").barh("Handedness")

---

<center> Return to Slides </center>

---

## Joins

We use join to **join** two tables on a column.

In [ ]:
drinks

In [ ]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

**Exercise:** Combine the `drinks` and `discounts` table into a new table which has all possible discounts for each drink.

In [ ]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

<details><summary>Click for Solution</summary>
    
```python
combined = drinks.join(column_label='Cafe', other=discounts, other_label='Location')
combined
```
    
</details>

**Exercise:** Calculate the lowest rate you can get for each drink using the available coupons.

In [ ]:
discount_frac = 1 - combined.column('Coupon % off')/100
# or you can do 
# discount_percentage = 100 -
combined.column('Coupon % off')
discount_frac

In [ ]:
(
    combined
    .with_column('Discounted Price', combined.column('Price') * discount_frac) # added new column with sale price
    .select('Drink', 'Discounted Price') # selected Drink and Discounted Price
    .group('Drink', min) # grouped by Drink and the cheapest price from Discounted Price
)

<details><summary>Click for Solution</summary>
    
```python
combined = drinks.join('Cafe', discounts, 'Location') # from previous question

discount_frac = 1-combined.column("Coupon % off")/100.0

(
    combined
    .with_column("Discounted Price", combined.column("Price") * discount_frac)
    .select("Drink", "Discounted Price")
    .group("Drink", min)
)

```
</details>

**Exercise:** What happens if I run the following? How many rows will it produce.

In [ ]:
drinks.join('Cafe', drinks, 'Cafe')

In [ ]:
drinks

In [ ]:
drinks.pivot('Cafe', 'Drink') # pivot allows us to categorize, in a visually efficient way
# our categorical variables
# inside the pivot (two-way) table, we see counts of items that satisfy the category grouping

In [ ]:
drinks.pivot('Cafe', 'Drink','Price',np.average)

---

<center> Return to Slides </center>

---

## Discussion Questions 

For the following practice questions we will use the sky scraper dataset

In [9]:
# From the CORGIS Dataset Project
# By Austin Cory Bart acbart@vt.edu
# Version 2.0.0, created 3/22/2016
# https://corgis-edu.github.io/corgis/csv/skyscrapers/

this_year = 2024
sky = Table.read_table("data/skyscrapers_v2.csv")
sky = (
    sky
    .with_column("age", this_year - sky.column("completed"))
    .drop("completed")
)
sky

name,material,city,height,age
One World Trade Center,mixed/composite,New York City,541.3,10
Willis Tower,steel,Chicago,442.14,50
432 Park Avenue,concrete,New York City,425.5,9
Trump International Hotel & Tower,concrete,Chicago,423.22,15
Empire State Building,steel,New York City,381,93
Bank of America Tower,mixed/composite,New York City,365.8,15
Stratosphere Tower,concrete,Las Vegas,350.22,28
Aon Center,steel,Chicago,346.26,51
John Hancock Center,steel,Chicago,343.69,55
Chrysler Building,steel,New York City,318.9,94


**Exercise:**  For each city, what’s the tallest building for each material?

In [10]:
sky.group('material')
sky.group('city')

city,count
Atlanta,38
Austin,17
Baltimore,14
Boston,32
Charlotte,12
Chicago,358
Cincinnati,13
Cleveland,16
Columbus,13
Dallas,20


In [11]:
# group by material: mixed/composite, steel, concrete AND city

# max height

tall_pivot = sky.pivot('material', 'city', 'height',max)
# .pivot('column label for column headings', 'column label for ROW headings', 
#        optional third argument for values, optional third argument for function to apply to values)
tall_pivot

city,concrete,mixed/composite,steel
Atlanta,27,4,7
Austin,16,0,1
Baltimore,8,0,6
Boston,4,2,26
Charlotte,5,3,4
Chicago,244,35,79
Cincinnati,5,1,7
Cleveland,2,3,11
Columbus,4,0,9
Dallas,7,2,11


<details><summary>Click for Solution</summary>
    
```python
tall_pivot = sky.pivot("material", "city", "height", max)
tall_pivot
```
</details>

In [ ]:
tall_pivot.barh("city")

**Exercise:** For each city that has at least 50 buildings in the dataset, compute the average height of buildings broken down by material.  Make a bar chart with the cities and the average height of buildings for each material.

<details><summary>Click for Solution</summary>
    
```python
num_buildings = sky.group("city")
(
    sky
    .join("city", num_buildings)
    .where("count", are.above_or_equal_to(50))
    .pivot("material", "city", "height", np.mean)
    .barh("city")
)
```
</details>

**Exercise:** Generate a table of the names and ages of the oldest buildings for each combination of city and materials.

<details><summary>Click for Solution</summary>
    
```python
def first(x):
    return x[0]
    
    
(
    sky
    .sort("age", descending=True) 
    .select("city", "material", "name", "age")
    .group(["city", "material"], first)
    #.where("city", "San Francisco") #<- for fun
)
```
    
</details>